In [5]:
import datetime as dt
import requests
from prefect.blocks.system import Secret
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
from mcpdb.tables import Asset, Provider, AssetType, ProviderAsset

In [6]:
postgresql_password: str = (await Secret.load("postgresql-password")).get()
host = "db-postgresql-lon1-65351-do-user-18535103-0.m.db.ondigitalocean.com"
port = 25060
database = "defaultdb"
user = "doadmin"
url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user=user,
    password=postgresql_password,
    host=host,
    port=port,
    database=database,
)
engine = create_engine(url)

In [7]:
keywords: list[str] = []
with Session(engine) as session:
    # Add crypto-related keywords
    crypto_keywords = ["cryptocurrency", "crypto", "blockchain"]
    keywords.extend(crypto_keywords)

    # Get all keywords from assets
    assets = (
        session.execute(
            select(Asset)
            .join(AssetType, Asset.asset_type_id == AssetType.id)
            .where(AssetType.name == "DIGITAL_CURRENCY")
        )
        .scalars()
        .all()
    )
    for asset in assets:
        keywords.append(asset.name)
        keywords.append(asset.description)

    # Get all keywords from provider assets
    provider_assets = session.execute(select(ProviderAsset)).scalars().all()
    for provider_asset in provider_assets:
        keywords.append(provider_asset.asset_code)

    # Normalize keywords
    keywords = [keyword.lower().strip() for keyword in keywords if keyword]
    keywords = list(set(keywords))  # Remove duplicates

keywords

['bitcoin',
 'ethereum',
 'xxbt',
 'blockchain',
 'eth',
 'xeth',
 'zusd',
 'btc',
 'cryptocurrency',
 'crypto']

In [8]:
categories: list[str] = []
with Session(engine) as session:
    # Get the coin desk provider.
    stmt = select(Provider).where(Provider.name == "CoinDesk")
    provider = session.execute(stmt).scalar_one_or_none()

    # Get all categories from provider assets.
    if provider:
        stmt = select(ProviderAsset).where(ProviderAsset.provider_id == provider.id)
        provider_assets = session.execute(stmt).scalars().all()
        for provider_asset in provider_assets:
            if provider_asset.asset_code:
                categories.append(provider_asset.asset_code)

categories

['BTC', 'ETH']

In [9]:
host = "https://data-api.coindesk.com"
uri = "/news/v1/article/list"
params = {
    "lang": "EN",
    "limit": 100,
    "to_ts": (dt.datetime.now() - dt.timedelta(hours=2)).timestamp(),
    "categories": ",".join(categories),
}
response = requests.get(f"{host}{uri}", params=params)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Coindesk API: {response.status_code}")
data = response.json()["Data"]
data

[{'TYPE': '121',
  'ID': 46651086,
  'GUID': 'https://ambcrypto.com/?p=504529',
  'PUBLISHED_ON': 1750726834,
  'PUBLISHED_ON_NS': None,
  'IMAGE_URL': 'https://images.cryptocompare.com/news/default/ambcrypto.png',
  'TITLE': 'Metaplanet buys $118M in Bitcoin during dip – Now holds over $1B in BTC',
  'SUBTITLE': None,
  'AUTHORS': 'Benjamin Njiri',
  'URL': 'https://ambcrypto.com/metaplanet-buys-118m-in-bitcoin-during-dip-now-holds-over-1b-in-btc/',
  'SOURCE_ID': 32,
  'BODY': "Metaplanet's latest purchase is just the latest in a series of them.",
  'KEYWORDS': 'Bitcoin|BTC Trading View|News|News 1|Social|Trading View',
  'LANG': 'EN',
  'UPVOTES': 0,
  'DOWNVOTES': 0,
  'SCORE': 0,
  'SENTIMENT': 'POSITIVE',
  'STATUS': 'ACTIVE',
  'CREATED_ON': 1750726917,
  'UPDATED_ON': None,
  'SOURCE_DATA': {'TYPE': '120',
   'ID': 32,
   'SOURCE_KEY': 'ambcrypto',
   'NAME': 'AMB Crypto',
   'IMAGE_URL': 'https://images.cryptocompare.com/news/default/ambcrypto.png',
   'URL': 'https://ambcrypt